In [1]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import Lossv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(Lossv2.Loss().to(device), (6, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

RuntimeError: Given input size: (2048x6x4). Calculated output size: (2048x1x0). Output size is too small

In [1]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import Lossv2
import generateLossImages
import time

global counter 
counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./models/rest/", best_model_dir="./models/best/"):
    global counter 
    f_path = checkpoint_dir + str(counter) + '_checkpoint.pt'
    counter = counter + 1
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

def load_ckp(model, optimizer, checkpoint_fpath="./models/best/best_model.pt"):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['index'], checkpoint['min_lr'], checkpoint['max_lr'], checkpoint['steps'], checkpoint['step_size'], checkpoint['falling'], checkpoint['threshold']

In [2]:
# in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

loss_fn = nn.L1Loss()
min_lr = 0.001
max_lr = 0.003
decay = 0.8
steps = 300
falling = True
start_epoch = 0
start_index = 0
momentum = 0.94
threshold = [0.16, 0.12, 0.09, 0.06, 0.04, 0.03, 0.02, 0.015, 0.01 -1]
step_size = (max_lr-min_lr)/steps
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=momentum)
scaler = torch.cuda.amp.GradScaler(enabled=True)
#torch.autograd.set_detect_anomaly(False)


In [2]:
loss_fn = nn.L1Loss()
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.96)
model, optimizer, start_epoch, start_index, min_lr, max_lr, steps, step_size, falling, threshold = load_ckp(model, optimizer)

In [1]:
li = [31.007527351379395, 0.01501011848449707, 0.0010006427764892578, 28.98952603340149, 0.48702239990234375, 47.146605014801025, 12.32689380645752, 0.5665144920349121]
su = sum(li)
print([100*l/su for l in li])


[25.72382749606882, 0.012452385971162576, 0.0008301326924880367, 24.049694721718666, 0.40403351288996003, 39.11279737602433, 10.22638426619226, 0.4699801084423085]


In [3]:
printing = True


running_loss = 0.
last_loss = 0.
lis = []
# might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100

times = [0]*10
transforms = 117
for epoch in range(start_epoch, epochs):
    training = generateLossImages.MakeIter(generateLossImages.get_image_pairs_transforms_with_loss(start=start_index if epoch == start_epoch else 0)) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
    val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training)
    for index, data in enumerate(training_loader): # loading data takes 25.72% of the time

        inputs, labels = data  #0.01%

        labels = labels.to("cuda") # 0.00%

        # Make predictions for this batch
        outputs = model(inputs) # 24.05%

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels) # 0.40%

        loss.backward() # 39.11%

        # Gather data and report
        running_loss += loss.item() # 10.23%

        if printing: # From here down only takes 0.47% of the time
            lis.append((f'{outputs.item():.3}',f'{labels.item():.3}'))
        if index % transforms == transforms-1:
            # Zero your gradients for every batch!
            # Adjust learning weights
            last_loss = running_loss / transforms # loss per batch
            if last_loss < threshold[0] and flags[0]:
                threshold = threshold[1:]
                flags = flags[1:]
                flag1 = False
                min_lr *= threshold_decay
                max_lr *= threshold_decay
                step_size = (max_lr-min_lr)/steps
                falling = True
                optimizer.param_groups[-1]['lr'] = max_lr
                if printing:
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                save_ckp(checkpoint, True)
            elif falling:
                optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
                if optimizer.param_groups[-1]['lr'] < min_lr:
                    falling = False
                    max_lr *= decay
                    min_lr *= decay
                    steps /= decay
                    step_size = (max_lr-min_lr)/steps

                    if printing:
                        print(min_lr)
                        print(max_lr)
                        print(steps)
                        print(step_size)
                    
                    
                    checkpoint = {
                        'epoch': epoch,
                        'index': index,
                        'min_lr': min_lr,
                        'max_lr': max_lr,
                        'steps': steps,
                        'step_size': step_size,
                        'falling': falling,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'threshold': threshold
                    }
                    save_ckp(checkpoint, False)

            else: 
                optimizer.param_groups[-1]['lr'] += step_size
                if optimizer.param_groups[-1]['lr'] > max_lr:
                    falling = True


            
            optimizer.step()
            optimizer.zero_grad()
            if printing:
                print(optimizer.param_groups[-1]['lr'])
                print('  batch {} loss: {}'.format(index + 1, last_loss))
                print(lis)
                lis = []
            running_loss = 0

            if last_loss < record:
                record = last_loss
                if last_loss < 0.08:
                    checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                    save_ckp(checkpoint, True)

torch.Size([3280, 4948, 3])


AttributeError: 'Tensor' object has no attribute 'astype'

In [5]:
print(index/4/50/117)

72.29034188034187


In [3]:
from torch import autocast
printing = False
running_loss = 0.
last_loss = 0.
lis = []
 # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100
times = [0]*8

for epoch in range(start_epoch, epochs):

    training = generateLossImages.MakeIter(generateLossImages.get_image_pairs_transforms_with_loss(start=start_index if epoch == start_epoch else 0)) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
    #val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training) # num_workers
    end = 0
    before_start = 0
    in_upstart = True
    for index, data in enumerate(training_loader):#[90.06174230575562, 0.03202486038208008, 0.016014575958251953, 84.36967897415161, 1.3647308349609375, 137.4005196094513, 34.81299662590027, 1.9427943229675293, 0, 0]
        before_start += 1
        if before_start > 117 and in_upstart:
            in_upstart = False
            times = [0]*8
        start = time.time()
        if end != 0:
            times[0] += (start-end)
        inputs, labels = data  

        load = time.time()
        times[1] += load-start

        lab = time.time()
        times[2] += (lab-load)

        # Make predictions for this batch
        outputs = model(inputs)

        out = time.time()
        times[3] += (out-lab)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)

        los = time.time()
        times[4] += (los-out)

        loss.backward()

        back = time.time()
        times[5] += (back-los)
        # Gather data and report
        #running_loss += loss.item()
        
        runn = time.time()
        times[6] += (runn-back)

        if printing:
            lis.append((f'{outputs.item():.3}',f'{labels.item():.3}'))
        if index % 117 == 116:
            # Zero your gradients for every batch!
            # Adjust learning weights
            #last_loss = running_loss / 117 # loss per batch
            #if last_loss < threshold[0] and flags[0]:
            #    threshold = threshold[1:]
            #    flags = flags[1:]
            #    flag1 = False
            #    min_lr *= threshold_decay
            #    max_lr *= threshold_decay
            #    step_size = (max_lr-min_lr)/steps
            #    falling = True
            #    optimizer.param_groups[-1]['lr'] = max_lr
            #    if printing:
            #        print(min_lr)
            #        print(max_lr)
            #        print(steps)
            #        print(step_size)
            #    checkpoint = {
            #        'epoch': epoch,
            #        'index': index,
            #        'min_lr': min_lr,
            #        'max_lr': max_lr,
            #        'steps': steps,
            #        'step_size': step_size,
            #        'falling': falling,
            #        'state_dict': model.state_dict(),
            #        'optimizer': optimizer.state_dict(),
            #        'threshold': threshold
            #    }
            #    save_ckp(checkpoint, True)
            #elif falling:
            if falling:
                optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
                if optimizer.param_groups[-1]['lr'] < min_lr:
                    falling = False
                    max_lr *= decay
                    min_lr *= decay
                    steps /= decay
                    step_size = (max_lr-min_lr)/steps

                    if printing:
                        print(min_lr)
                        print(max_lr)
                        print(steps)
                        print(step_size)
                    checkpoint = {
                        'epoch': epoch,
                        'index': index,
                        'min_lr': min_lr,
                        'max_lr': max_lr,
                        'steps': steps,
                        'step_size': step_size,
                        'falling': falling,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'threshold': threshold
                    }
                    save_ckp(checkpoint, True)

            else: 
                optimizer.param_groups[-1]['lr'] += step_size
                if optimizer.param_groups[-1]['lr'] > max_lr:
                    falling = True


            
            optimizer.step()
            optimizer.zero_grad()
        #    if printing:
        #        print(optimizer.param_groups[-1]['lr'])
        #        print('  batch {} loss: {}'.format(index + 1, last_loss))
        #        print(lis)
        #        lis = []
        #    running_loss = 0

        #    if last_loss < record:
        #        record = last_loss
        #        if last_loss < 0.08:
        #            checkpoint = {
        #            'epoch': epoch,
        #            'index': index,
        #            'min_lr': min_lr,
        #            'max_lr': max_lr,
        #            'steps': steps,
        #            'step_size': step_size,
        #            'falling': falling,
        #            'state_dict': model.state_dict(),
        #            'optimizer': optimizer.state_dict(),
        #            'threshold': threshold
        #        }
        #            save_ckp(checkpoint, True)
            #print(times)
        end = time.time()
        times[7] += (end-runn)
        if before_start > 117*4:
            break
    
    break
                

c:\Users\Rani\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [4]:
print(times)
su = sum(times)
print([100*l/su for l in times])


[0.6015474796295166, 0.002002239227294922, 0.0, 7.0254199504852295, 0.07607626914978027, 10.602812767028809, 0.0010006427764892578, 0.01801609992980957]
[3.282324263777951, 0.010925153242576506, 0.0, 38.3339755340833, 0.4151076890585865, 57.85390312157438, 0.005459974774838484, 0.09830426348836553]
